In [279]:
import sys
import os

# Get the current working directory
current_dir = os.getcwd()

# Add the parent directory to sys.path
parent_dir = os.path.abspath(os.path.join(current_dir, '..'))
sys.path.insert(0, parent_dir)

In [280]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter
from urllib.request import urlretrieve
from scipy.stats import pearsonr

from config.config import DATA_PATH

In [281]:
pd.set_option('display.max_columns', None)

df = pd.read_csv(DATA_PATH + '/preprocessed/airline_performance.csv', parse_dates=['scheduled_departure_datetime', 'scheduled_arrival_datetime'])

In [282]:
df.head()

,year,quarter,month,day_of_month,day_of_week,marketing_airline_id,flight_number_marketing_airline,operating_airline_id,tail_number,origin_airport_id,origin_city_market_id,origin,origin_state,dest_airport_id,dest_city_market_id,dest,dest_state,dep_delay,dep_del15,departure_delay_groups,taxi_out,taxi_in,arr_delay,arr_del15,arrival_delay_groups,cancelled,diverted,scheduled_elapsed_time,actual_elapsed_time,air_time,distance,distance_group,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,div_airport_landings,code_share_flight,origin_city,destination_city,scheduled_departure_datetime,scheduled_arrival_datetime,actual_departure_datetime,actual_arrival_datetime,overnight_depature,overnight_arrival,airline_mkt,airline_ops,origin_timezone,destination_timezone,scheduled_departure_datetime_utc,actual_departure_datetime_utc,scheduled_arrival_datetime_utc,actual_arrival_datetime_utc,is_holiday,dep_delay_clipped,arr_delay_clipped,route_id,hour_of_day,departure_window,dep_delay_mean_10D,dep_delay_median_10D,dep_delay_max_10D,cancelled_sum_10D,div_airport_landings_sum_10D,n_flights_10D,dep_delay_mean_30D,dep_delay_median_30D,dep_delay_max_30D,cancelled_sum_30D,div_airport_landings_sum_30D,dep_delay_max_90D,cancelled_sum_90D,div_airport_landings_sum_90D,departure_status,arrival_status,region_origin,division_origin,region_dest,division_dest,temperature_2m_origin,precipitation_origin,snow_depth_origin,weather_code_origin,visibility_origin,wind_speed_10m_origin,wind_direction_10m_origin,wind_gusts_10m_origin,temperature_2m_dest,precipitation_dest,snow_depth_dest,weather_code_dest,visibility_dest,wind_speed_10m_dest,wind_direction_10m_dest,wind_gusts_10m_dest,origin_weather_impact_category,dest_weather_impact_category
0,2023,3,8,6,7,20416,2252,20416,N978NK,12889,32211,LAS,NV,11057,31057,CLT,NC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,263.0,NaN,NaN,1916.0,8,NaN,NaN,NaN,NaN,NaN,0.0,0,Las Vegas,Charlotte,2023-08-06 23:59:00,2023-08-07 07:22:00,NaN,NaN,0,0,Spirit Air Lines,Spirit Air Lines,America/Los_Angeles,America/New_York,2023-08-07 07:00:00+00:00,NaN,2023-08-07 11:00:00+00:00,NaN,0,NaN,NaN,3228,23,late night,5.125000,2.0,31.0,3.0,0.0,8.0,15.518519,5.0,82.0,4.0,0.0,163.0,5.0,0.0,cancelled,cancelled,West,Mountain,South,South Atlantic,87.6,0.0,0.0,0,293635.188,5.1,180,8.5,72.6,0.0,0.0,3,40026.246,4.2,196,8.5,Low Impact,Low Impact
1,2023,3,8,7,1,20416,2252,20416,N974NK,12889,32211,LAS,NV,11057,31057,CLT,NC,76.0,1.0,5.0,19.0,9.0,69.0,1.0,4.0,0,0,263.0,256.0,228.0,1916.0,8,3.0,0.0,1.0,0.0,65.0,0.0,0,Las Vegas,Charlotte,2023-08-07 23:59:00,2023-08-08 07:22:00,2023-08-08 01:15:00,2023-08-08 08:31:00,0,0,Spirit Air Lines,Spirit Air Lines,America/Los_Angeles,America/New_York,2023-08-08 07:00:00+00:00,2023-08-08 08:15:00+00:00,2023-08-08 11:00:00+00:00,2023-08-08 12:31:00+00:00,0,76.0,69.0,3228,23,late night,14.000000,2.0,76.0,3.0,0.0,8.0,17.740741,5.0,82.0,4.0,0.0,163.0,5.0,0.0,delayed,delayed,West,Mountain,South,South Atlantic,89.7,0.0,0.0,0,293635.188,13.5,220,25.3,70.0,0.0,0.0,3,42322.836,4.9,223,18.1,Low Impact,Low Impact
2,2023,3,8,9,3,20416,2252,20416,N519NK,12889,32211,LAS,NV,11057,31057,CLT,NC,-11.0,0.0,-1.0,14.0,10.0,-13.0,0.0,-1.0,0,0,258.0,256.0,232.0,1916.0,8,NaN,NaN,NaN,NaN,NaN,0.0,0,Las Vegas,Charlotte,2023-08-09 23:10:00,2023-08-10 06:28:00,2023-08-09 22:59:00,2023-08-10 06:15:00,0,0,Spirit Air Lines,Spirit Air Lines,America/Los_Angeles,America/New_York,2023-08-10 06:00:00+00:00,2023-08-10 05:59:00+00:00,2023-08-10 10:00:00+00:00,2023-08-10 10:15:00+00:00,0,-11.0,-13.0,3228,23,late night,12.375000,1.5,76.0,3.0,0.0,8.0,15.185185,5.0,82.0,4.0,0.0,163.0,5.0,0.0,early,early,West,Mountain,South,South Atlantic,91.9,0.0,0.0,3,234251.969,11.7,239,22.6,72.9,0.0,0.0,3,52821.523,3.4,212,9.4,Low Impact,Low Impact
3,2023,3,8,10,4,20416,2252,20416,N532NK,12889,32211,LAS,NV,11057,31057,CLT,NC,-8.0,0.0,-1.0,12.0,7.0,-30.0,0.0,-2.0,0,0,258.0,236.0,217.0,1916.0,8,NaN,NaN,NaN,NaN,NaN,0.0,0,Las Vegas,Charlotte,2023-08-10 23:10:00,2023-08-11 06

In [283]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1493764 entries, 0 to 1493763
Data columns (total 99 columns):
 #   Column                            Non-Null Count    Dtype         
---  ------                            --------------    -----         
 0   year                              1493764 non-null  int64         
 1   quarter                           1493764 non-null  int64         
 2   month                             1493764 non-null  int64         
 3   day_of_month                      1493764 non-null  int64         
 4   day_of_week                       1493764 non-null  int64         
 5   marketing_airline_id              1493764 non-null  int64         
 6   flight_number_marketing_airline   1493764 non-null  int64         
 7   operating_airline_id              1493764 non-null  int64         
 8   tail_number                       1488178 non-null  object        
 9   origin_airport_id                 1493764 non-null  int64         
 10  origin_city_market

In [284]:
df.shape

(1493764, 99)

In [285]:
df.columns.to_list()

['year',
 'quarter',
 'month',
 'day_of_month',
 'day_of_week',
 'marketing_airline_id',
 'flight_number_marketing_airline',
 'operating_airline_id',
 'tail_number',
 'origin_airport_id',
 'origin_city_market_id',
 'origin',
 'origin_state',
 'dest_airport_id',
 'dest_city_market_id',
 'dest',
 'dest_state',
 'dep_delay',
 'dep_del15',
 'departure_delay_groups',
 'taxi_out',
 'taxi_in',
 'arr_delay',
 'arr_del15',
 'arrival_delay_groups',
 'cancelled',
 'diverted',
 'scheduled_elapsed_time',
 'actual_elapsed_time',
 'air_time',
 'distance',
 'distance_group',
 'carrier_delay',
 'weather_delay',
 'nas_delay',
 'security_delay',
 'late_aircraft_delay',
 'div_airport_landings',
 'code_share_flight',
 'origin_city',
 'destination_city',
 'scheduled_departure_datetime',
 'scheduled_arrival_datetime',
 'actual_departure_datetime',
 'actual_arrival_datetime',
 'overnight_depature',
 'overnight_arrival',
 'airline_mkt',
 'airline_ops',
 'origin_timezone',
 'destination_timezone',
 'scheduled_d

## Columns to Drop


In [286]:
cols_to_keep = [
    'quarter',
    'month',
    'day_of_month',
    'day_of_week',
    'hour_of_day',
    'origin',
    'dest',
    'cancelled',
    'scheduled_elapsed_time',
    'distance',
    'code_share_flight',
    'scheduled_departure_datetime',
    'scheduled_arrival_datetime',
    'airline_mkt',
    'airline_ops',
    'is_holiday',
    'route_id',
    'departure_window',
    'dep_delay_mean_10D',
    'dep_delay_median_10D',
    'dep_delay_max_10D',
    'cancelled_sum_10D',
    'div_airport_landings_sum_10D',
    'n_flights_10D',
    'dep_delay_mean_30D',
    'dep_delay_median_30D',
    'dep_delay_max_30D',
    'cancelled_sum_30D',
    'div_airport_landings_sum_30D',
    'cancelled_sum_90D',
    'div_airport_landings_sum_90D',
    'region_origin',
    'division_origin',
    'region_dest',
    'division_dest',
    'temperature_2m_origin',
    'precipitation_origin',
    'snow_depth_origin',
    'visibility_origin',
    'wind_speed_10m_origin',
    'wind_direction_10m_origin',
    'wind_gusts_10m_origin',
    'temperature_2m_dest',
    'precipitation_dest',
    'snow_depth_dest',
    'visibility_dest',
    'wind_speed_10m_dest',
    'wind_direction_10m_dest',
    'wind_gusts_10m_dest',
    'origin_weather_impact_category',
    'dest_weather_impact_category']

flights_top20 = df[cols_to_keep]

## Missing Values

In [287]:
missing_values = flights_top20.isna().sum().reset_index(name='count')
missing_values[missing_values['count'] > 0].sort_values('count', ascending=False).head(60)

,index,count
18,dep_delay_mean_10D,188
19,dep_delay_median_10D,188
20,dep_delay_max_10D,188
24,dep_delay_mean_30D,166
25,dep_delay_median_30D,166
26,dep_delay_max_30D,166
8,scheduled_elapsed_time,1


In [288]:
flights_top20[['cancelled']].sum() / len(flights_top20)

cancelled    0.014416
dtype: float64

In [289]:
flights_missing_dropped = flights_top20.dropna(subset=['wind_gusts_10m_dest'])
missing_values = flights_missing_dropped.isna().sum().reset_index(name='count')
missing_values[missing_values['count'] > 0].sort_values('count', ascending=False).head(60)

,index,count
18,dep_delay_mean_10D,188
19,dep_delay_median_10D,188
20,dep_delay_max_10D,188
24,dep_delay_mean_30D,166
25,dep_delay_median_30D,166
26,dep_delay_max_30D,166
8,scheduled_elapsed_time,1


In [290]:
# calculate the average aggregated performance by airline_mkt and route_id
route_airline_agg = flights_top20.groupby(['airline_mkt', 'route_id']).agg({
    'dep_delay_mean_10D':'mean',
    'dep_delay_median_10D':'mean',
    'dep_delay_max_10D':'mean',
    'cancelled_sum_10D':'mean',
    'div_airport_landings_sum_10D':'mean',
    'n_flights_10D':'mean',
    'dep_delay_mean_30D':'mean',
    'dep_delay_median_30D':'mean',
    'dep_delay_max_30D':'mean',
    'cancelled_sum_30D':'mean',
    'div_airport_landings_sum_30D':'mean',
    'cancelled_sum_90D':'mean',
    'div_airport_landings_sum_90D':'mean',
    
}).reset_index()

agg_cols = route_airline_agg.drop(columns=['airline_mkt', 'route_id']).columns.to_list()
agg_cols
# merge the aggregated data back into the missing stats to fill missing values
missing_stats_filled = pd.merge(
    flights_top20,
    route_airline_agg,
    on=['airline_mkt', 'route_id'],
    suffixes=('', '_agg'),
    how='left'
)

# Fill missing values in the original columns with the aggregated values
for col in agg_cols:
    missing_stats_filled[col] = missing_stats_filled[col].fillna(missing_stats_filled[col + '_agg'])

# Drop the extra columns used for merging
missing_stats_filled = missing_stats_filled.drop(columns=[col + '_agg' for col in agg_cols])
missing_stats_filled.head()

,quarter,month,day_of_month,day_of_week,hour_of_day,origin,dest,cancelled,scheduled_elapsed_time,distance,code_share_flight,scheduled_departure_datetime,scheduled_arrival_datetime,airline_mkt,airline_ops,is_holiday,route_id,departure_window,dep_delay_mean_10D,dep_delay_median_10D,dep_delay_max_10D,cancelled_sum_10D,div_airport_landings_sum_10D,n_flights_10D,dep_delay_mean_30D,dep_delay_median_30D,dep_delay_max_30D,cancelled_sum_30D,div_airport_landings_sum_30D,cancelled_sum_90D,div_airport_landings_sum_90D,region_origin,division_origin,region_dest,division_dest,temperature_2m_origin,precipitation_origin,snow_depth_origin,visibility_origin,wind_speed_10m_origin,wind_direction_10m_origin,wind_gusts_10m_origin,temperature_2m_dest,precipitation_dest,snow_depth_dest,visibility_dest,wind_speed_10m_dest,wind_direction_10m_dest,wind_gusts_10m_dest,origin_weather_impact_category,dest_weather_impact_category
0,3,8,6,7,23,LAS,CLT,1,263.0,1916.0,0,2023-08-06 23:59:00,2023-08-07 07:22:00,Spirit Air Lines,Spirit Air Lines,0,3228,late night,5.125000,2.0,31.0,3.0,0.0,8.0,15.518519,5.0,82.0,4.0,0.0,5.0,0.0,West,Mountain,South,South Atlantic,87.6,0.0,0.0,293635.188,5.1,180,8.5,72.6,0.0,0.0,40026.246,4.2,196,8.5,Low Impact,Low Impact
1,3,8,7,1,23,LAS,CLT,0,263.0,1916.0,0,2023-08-07 23:59:00,2023-08-08 07:22:00,Spirit Air Lines,Spirit Air Lines,0,3228,late night,14.000000,2.0,76.0,3.0,0.0,8.0,17.740741,5.0,82.0,4.0,0.0,5.0,0.0,West,Mountain,South,South Atlantic,89.7,0.0,0.0,293635.188,13.5,220,25.3,70.0,0.0,0.0,42322.836,4.9,223,18.1,Low Impact,Low Impact
2,3,8,9,3,23,LAS,CLT,0,258.0,1916.0,0,2023-08-09 23:10:00,2023-08-10 06:28:00,Spirit Air Lines,Spirit Air Lines,0,3228,late night,12.375000,1.5,76.0,3.0,0.0,8.0,15.185185,5.0,82.0,4.0,0.0,5.0,0.0,West,Mountain,South,South Atlantic,91.9,0.0,0.0,234251.969,11.7,239,22.6,72.9,0.0,0.0,52821.523,3.4,212,9.4,Low Impact,Low Impact
3,3,8,10,4,23,LAS,CLT,0,258.0,1916.0,0,2023-08-10 23:10:00,2023-08-11 06:28:00,Spirit Air Lines,Spirit Air Lines,0,3228,late night,10.111111,1.0,76.0,2.0,0.0,9.0,11.851852,3.0,76.0,4.0,0.0,5.0,0.0,West,Mountain,South,South Atlantic,87.6,0.0,0.0,173884.516,5.9,18,9.4,68.7,0.0,0.0,44947.508,1.1,90,1.6,Low Impact,Low Impact
4,3,8,12,6,23,LAS,CLT,0,258.0,1916.0,0,2023-08-12 23:10:00,2023-08-13 06:28:00,Spirit Air Lines,Spirit Air Lines,0,3228,late night,12.250000,3.0,76.0,1.0,0.0,8.0,12.961538,4.5,76.0,4.0,0.0,5.0,0.0,West,Mountain,South,South Atlantic,92.2,0.0,0.0,265091.875,10.2,50,13.0,69.0,0.0,0.0,40354.332,3.1,176,6.5,Low Impact,Low Impact


In [291]:
missing_values = missing_stats_filled.isna().sum().reset_index(name='count')
missing_values[missing_values['count'] > 0].sort_values('count', ascending=False).head(60)

,index,count
8,scheduled_elapsed_time,1


In [292]:
missing_time = missing_stats_filled[missing_stats_filled['scheduled_elapsed_time'].isna()]
missing_time['scheduled_elapsed_time'] = (pd.to_datetime(missing_time['scheduled_arrival_datetime']) - pd.to_datetime(missing_time['scheduled_departure_datetime'])).dt.total_seconds() / 60
missing_time

/var/folders/b2/nkpmrw0s42q_zm32rwshljyr0000gn/T/ipykernel_9432/1857774745.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_time['scheduled_elapsed_time'] = (pd.to_datetime(missing_time['scheduled_arrival_datetime']) - pd.to_datetime(missing_time['scheduled_departure_datetime'])).dt.total_seconds() / 60


,quarter,month,day_of_month,day_of_week,hour_of_day,origin,dest,cancelled,scheduled_elapsed_time,distance,code_share_flight,scheduled_departure_datetime,scheduled_arrival_datetime,airline_mkt,airline_ops,is_holiday,route_id,departure_window,dep_delay_mean_10D,dep_delay_median_10D,dep_delay_max_10D,cancelled_sum_10D,div_airport_landings_sum_10D,n_flights_10D,dep_delay_mean_30D,dep_delay_median_30D,dep_delay_max_30D,cancelled_sum_30D,div_airport_landings_sum_30D,cancelled_sum_90D,div_airport_landings_sum_90D,region_origin,division_origin,region_dest,division_dest,temperature_2m_origin,precipitation_origin,snow_depth_origin,visibility_origin,wind_speed_10m_origin,wind_direction_10m_origin,wind_gusts_10m_origin,temperature_2m_dest,precipitation_dest,snow_depth_dest,visibility_dest,wind_speed_10m_dest,wind_direction_10m_dest,wind_gusts_10m_dest,origin_weather_impact_category,dest_weather_impact_category
1421876,1,3,19,7,10,SFO,LAX,1,89.0,337.0,1,2023-03-19 10:20:00,2023-03-19 11:49:00,Alaska Airlines Inc.,SkyWest Airlines Inc.,0,5963,morning,4.296296,-2.0,69.0,3.0,0.0,27.0,5.776471,-3.0,209.0,4.0,0.0,6.0,0.0,West,Pacific,West,Pacific,50.8,0.0,0.0,43635.172,2.7,90,11.6,59.6,0.0,0.0,67585.305,4.8,221,11.4,Low Impact,Low Impact


In [293]:
missing_stats_filled['scheduled_elapsed_time'] = (pd.to_datetime(missing_stats_filled['scheduled_arrival_datetime']) - pd.to_datetime(missing_stats_filled['scheduled_departure_datetime'])).dt.total_seconds() / 60

In [294]:
missing_values = missing_stats_filled.isna().sum().reset_index(name='count')
missing_values[missing_values['count'] > 0].sort_values('count', ascending=False)

,index,count


In [295]:
missing_stats_filled.head()

,quarter,month,day_of_month,day_of_week,hour_of_day,origin,dest,cancelled,scheduled_elapsed_time,distance,code_share_flight,scheduled_departure_datetime,scheduled_arrival_datetime,airline_mkt,airline_ops,is_holiday,route_id,departure_window,dep_delay_mean_10D,dep_delay_median_10D,dep_delay_max_10D,cancelled_sum_10D,div_airport_landings_sum_10D,n_flights_10D,dep_delay_mean_30D,dep_delay_median_30D,dep_delay_max_30D,cancelled_sum_30D,div_airport_landings_sum_30D,cancelled_sum_90D,div_airport_landings_sum_90D,region_origin,division_origin,region_dest,division_dest,temperature_2m_origin,precipitation_origin,snow_depth_origin,visibility_origin,wind_speed_10m_origin,wind_direction_10m_origin,wind_gusts_10m_origin,temperature_2m_dest,precipitation_dest,snow_depth_dest,visibility_dest,wind_speed_10m_dest,wind_direction_10m_dest,wind_gusts_10m_dest,origin_weather_impact_category,dest_weather_impact_category
0,3,8,6,7,23,LAS,CLT,1,443.0,1916.0,0,2023-08-06 23:59:00,2023-08-07 07:22:00,Spirit Air Lines,Spirit Air Lines,0,3228,late night,5.125000,2.0,31.0,3.0,0.0,8.0,15.518519,5.0,82.0,4.0,0.0,5.0,0.0,West,Mountain,South,South Atlantic,87.6,0.0,0.0,293635.188,5.1,180,8.5,72.6,0.0,0.0,40026.246,4.2,196,8.5,Low Impact,Low Impact
1,3,8,7,1,23,LAS,CLT,0,443.0,1916.0,0,2023-08-07 23:59:00,2023-08-08 07:22:00,Spirit Air Lines,Spirit Air Lines,0,3228,late night,14.000000,2.0,76.0,3.0,0.0,8.0,17.740741,5.0,82.0,4.0,0.0,5.0,0.0,West,Mountain,South,South Atlantic,89.7,0.0,0.0,293635.188,13.5,220,25.3,70.0,0.0,0.0,42322.836,4.9,223,18.1,Low Impact,Low Impact
2,3,8,9,3,23,LAS,CLT,0,438.0,1916.0,0,2023-08-09 23:10:00,2023-08-10 06:28:00,Spirit Air Lines,Spirit Air Lines,0,3228,late night,12.375000,1.5,76.0,3.0,0.0,8.0,15.185185,5.0,82.0,4.0,0.0,5.0,0.0,West,Mountain,South,South Atlantic,91.9,0.0,0.0,234251.969,11.7,239,22.6,72.9,0.0,0.0,52821.523,3.4,212,9.4,Low Impact,Low Impact
3,3,8,10,4,23,LAS,CLT,0,438.0,1916.0,0,2023-08-10 23:10:00,2023-08-11 06:28:00,Spirit Air Lines,Spirit Air Lines,0,3228,late night,10.111111,1.0,76.0,2.0,0.0,9.0,11.851852,3.0,76.0,4.0,0.0,5.0,0.0,West,Mountain,South,South Atlantic,87.6,0.0,0.0,173884.516,5.9,18,9.4,68.7,0.0,0.0,44947.508,1.1,90,1.6,Low Impact,Low Impact
4,3,8,12,6,23,LAS,CLT,0,438.0,1916.0,0,2023-08-12 23:10:00,2023-08-13 06:28:00,Spirit Air Lines,Spirit Air Lines,0,3228,late night,12.250000,3.0,76.0,1.0,0.0,8.0,12.961538,4.5,76.0,4.0,0.0,5.0,0.0,West,Mountain,South,South Atlantic,92.2,0.0,0.0,265091.875,10.2,50,13.0,69.0,0.0,0.0,40354.332,3.1,176,6.5,Low Impact,Low Impact


In [296]:
flights_top20 = missing_stats_filled.drop(columns='route_id')

In [297]:
flights_top20

,quarter,month,day_of_month,day_of_week,hour_of_day,origin,dest,cancelled,scheduled_elapsed_time,distance,code_share_flight,scheduled_departure_datetime,scheduled_arrival_datetime,airline_mkt,airline_ops,is_holiday,departure_window,dep_delay_mean_10D,dep_delay_median_10D,dep_delay_max_10D,cancelled_sum_10D,div_airport_landings_sum_10D,n_flights_10D,dep_delay_mean_30D,dep_delay_median_30D,dep_delay_max_30D,cancelled_sum_30D,div_airport_landings_sum_30D,cancelled_sum_90D,div_airport_landings_sum_90D,region_origin,division_origin,region_dest,division_dest,temperature_2m_origin,precipitation_origin,snow_depth_origin,visibility_origin,wind_speed_10m_origin,wind_direction_10m_origin,wind_gusts_10m_origin,temperature_2m_dest,precipitation_dest,snow_depth_dest,visibility_dest,wind_speed_10m_dest,wind_direction_10m_dest,wind_gusts_10m_dest,origin_weather_impact_category,dest_weather_impact_category
0,3,8,6,7,23,LAS,CLT,1,443.0,1916.0,0,2023-08-06 23:59:00,2023-08-07 07:22:00,Spirit Air Lines,Spirit Air Lines,0,late night,5.125000,2.0,31.0,3.0,0.0,8.0,15.518519,5.0,82.0,4.0,0.0,5.0,0.0,West,Mountain,South,South Atlantic,87.6,0.0,0.0,293635.188,5.1,180,8.5,72.6,0.0,0.0,40026.246,4.2,196,8.5,Low Impact,Low Impact
1,3,8,7,1,23,LAS,CLT,0,443.0,1916.0,0,2023-08-07 23:59:00,2023-08-08 07:22:00,Spirit Air Lines,Spirit Air Lines,0,late night,14.000000,2.0,76.0,3.0,0.0,8.0,17.740741,5.0,82.0,4.0,0.0,5.0,0.0,West,Mountain,South,South Atlantic,89.7,0.0,0.0,293635.188,13.5,220,25.3,70.0,0.0,0.0,42322.836,4.9,223,18.1,Low Impact,Low Impact
2,3,8,9,3,23,LAS,CLT,0,438.0,1916.0,0,2023-08-09 23:10:00,2023-08-10 06:28:00,Spirit Air Lines,Spirit Air Lines,0,late night,12.375000,1.5,76.0,3.0,0.0,8.0,15.185185,5.0,82.0,4.0,0.0,5.0,0.0,West,Mountain,South,South Atlantic,91.9,0.0,0.0,234251.969,11.7,239,22.6,72.9,0.0,0.0,52821.523,3.4,212,9.4,Low Impact,Low Impact
3,3,8,10,4,23,LAS,CLT,0,438.0,1916.0,0,2023-08-10 23:10:00,2023-08-11 06:28:00,Spirit Air Lines,Spirit Air Lines,0,late night,10.111111,1.0,76.0,2.0,0.0,9.0,11.851852,3.0,76.0,4.0,0.0,5.0,0.0,West,Mountain,South,South Atlantic,87.6,0.0,0.0,173884.516,5.9,18,9.4,68.7,0.0,0.0,44947.508,1.1,90,1.6,Low Impact,Low Impact
4,3,8,12,6,23,LAS,CLT,0,438.0,1916.0,0,2023-08-12 23:10:00,2023-08-13 06:28:00,Spirit Air Lines,Spirit Air Lines,0,late night,12.250000,3.0,76.0,1.0,0.0,8.0,12.961538,4.5,76.0,4.0,0.0,5.0,0.0,West,Mountain,South,South Atlantic,92.2,0.0,0.0,265091.875,10.2,50,13.0,69.0,0.0,0.0,40354.332,3.1,176,6.5,Low Impact,Low Impact
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1493759,1,3,27,1,19,ATL,CLT,0,84.0,226.0,1,2023-03-27 19:45:00,2023-03-27 21:09:00,American Airlines Inc.,PSA Airlines Inc.,0,night,5.222222,-6.0,58.0,0.0,0.0,9.0,6.821429,-6.0,95.0,0.0,0.0,2.0,0.0,South,South Atlantic,South,South Atlantic,72.9,0.0,0.0,168307.094,4.9,273,7.2,68.6,0.0,0.0,72506.562,1.1,281,6.3,Low Impact,Low Impact
1493760,1,3,29,3,19,ATL,CLT,0,84.0,226.0,1,2023-03-29 19:45:00,2023-03-29 21:09:00,American Airlines Inc.,PSA Airlines Inc.,0,night,5.625000,-6.5,58.0,0.0,0.0,8.0,3.555556,-6.0,75.0,0.0,0.0,2.0,0.0,South,South Atlantic,South,South Atlantic,59.8,0.0,0.0,225065.625,3.2,245,5.8,54.1,0.0,0.0,147309.719,1.8,187,3.8,Low Impact,Low Impact
1493761,1,3,30,4,19,ATL,CLT,0,84.0,226.0,1,2023-03-30 19:45:00,2023-03-30 21:09:00,American Airlines Inc.,PSA Airlines Inc.,0,night,1.125000,-8.5,58.0,0.0,0.0,8.0,3.407407,-6.0,75.0,0.0,0.0,2.0,0.0,South,South Atlantic,South,South Atlantic,67.3,0.0,0.0,215879.266,3.2,124,10.5,59.7,0.0,0.0,219488.188,0.7,90,2.5,Low Impact,Low Impact
1493762,1,3,31,5,19,ATL,CLT,0,84.0,226.0,1,2023-03-31 19:45:00,2023-03-31 21:09:00,American Airlines Inc.,PSA Airlines Inc.,0,night,0.333333,-7.0,58.0,0.0,0.0,9.0,3.846154,-6.0,75.0,0.0,0.0,2.0,0.0,South,South Atlantic,South,South Atlantic,73.2,0.0,0.0,127952.758,3.6,176,19.7,65.0,0.0,0.0,11

## Train / Test Split

In [298]:
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.feature_selection import SelectKBest, f_classif

In [474]:
X = flights_top20.drop(columns='cancelled')
y = flights_top20[['cancelled']]

In [475]:
# use StratifiedShuffleSplit to ensure equal class imbalances
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=42)

# Perform the stratified split
for train_index, test_index in sss.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

In [476]:
y_train['cancelled'].mean()

0.014416134134888499

In [477]:
y_test['cancelled'].mean()

0.014415459799611719

In [478]:
train = pd.concat([X_train, y_train], axis=1)
test = pd.concat([X_test, y_test], axis=1)

In [479]:
test.to_csv(DATA_PATH + '/model_splits/test_data.csv', index=False)

## Encoding Categorical Variables

### Ordinal Encoding for Weather Codes

In [480]:
weather_category_ordinal = {'Low Impact': 1, 'Moderate Impact': 2, 'High Impact': 3, 'Extreme Impact': 4}

train['origin_weather_impact_category'] = train['origin_weather_impact_category'].map(weather_category_ordinal)
train['dest_weather_impact_category'] = train['dest_weather_impact_category'].map(weather_category_ordinal)

test['origin_weather_impact_category'] = test['origin_weather_impact_category'].map(weather_category_ordinal)
test['dest_weather_impact_category'] = test['dest_weather_impact_category'].map(weather_category_ordinal)

In [481]:
train.head()

,quarter,month,day_of_month,day_of_week,hour_of_day,origin,dest,scheduled_elapsed_time,distance,code_share_flight,scheduled_departure_datetime,scheduled_arrival_datetime,airline_mkt,airline_ops,is_holiday,departure_window,dep_delay_mean_10D,dep_delay_median_10D,dep_delay_max_10D,cancelled_sum_10D,div_airport_landings_sum_10D,n_flights_10D,dep_delay_mean_30D,dep_delay_median_30D,dep_delay_max_30D,cancelled_sum_30D,div_airport_landings_sum_30D,cancelled_sum_90D,div_airport_landings_sum_90D,region_origin,division_origin,region_dest,division_dest,temperature_2m_origin,precipitation_origin,snow_depth_origin,visibility_origin,wind_speed_10m_origin,wind_direction_10m_origin,wind_gusts_10m_origin,temperature_2m_dest,precipitation_dest,snow_depth_dest,visibility_dest,wind_speed_10m_dest,wind_direction_10m_dest,wind_gusts_10m_dest,origin_weather_impact_category,dest_weather_impact_category,cancelled
783949,2,4,30,7,7,IAH,BOS,288.0,1597.0,0,2023-04-30 07:42:00,2023-04-30 12:30:00,United Air Lines Inc.,United Air Lines Inc.,0,morning,2.000000,-3.0,29.0,0.0,0.0,10.0,9.700000,-2.0,203.0,0.0,0.0,0.0,0.0,South,West South Central,Northeast,New England,55.7,0.0,0.0,51509.188,2.3,241,12.8,49.5,0.016,0.0,656.168,15.2,36,25.1,1,2,0
1424121,1,3,31,5,20,IAH,EWR,277.0,1400.0,0,2023-03-31 20:48:00,2023-04-01 01:25:00,United Air Lines Inc.,United Air Lines Inc.,0,night,20.545455,13.0,76.0,0.0,0.0,11.0,11.562500,2.5,76.0,0.0,0.0,0.0,0.0,South,West South Central,Northeast,Middle Atlantic,76.1,0.0,0.0,67257.219,7.9,175,19.2,53.0,0.000,0.0,76115.484,2.6,52,4.9,1,1,0
1135132,1,1,30,1,14,SFO,JFK,502.0,2586.0,0,2023-01-30 14:00:00,2023-01-30 22:22:00,Delta Air Lines Inc.,Delta Air Lines Inc.,0,early afternoon,-1.166667,-4.5,9.0,0.0,0.0,6.0,-0.250000,-1.5,16.0,0.0,0.0,0.0,0.0,West,Pacific,Northeast,Middle Atlantic,51.9,0.0,0.0,192257.219,6.1,86,13.2,42.5,0.000,0.0,64960.629,2.9,261,9.2,1,1,0
19004,3,8,14,1,18,IAH,MCO,204.0,854.0,0,2023-08-14 18:20:00,2023-08-14 21:44:00,United Air Lines Inc.,United Air Lines Inc.,0,evening,8.100000,2.0,49.0,0.0,0.0,10.0,35.428571,4.5,209.0,2.0,0.0,5.0,0.0,South,West South Central,South,South Atlantic,102.9,0.0,0.0,190288.719,0.2,90,4.0,82.6,0.000,0.0,61679.789,3.0,163,5.8,1,1,0
1109133,4,11,30,4,13,ATL,MSP,94.0,907.0,0,2023-11-30 13:48:00,2023-11-30 15:22:00,Delta Air Lines Inc.,Delta Air Lines Inc.,0,early afternoon,7.900000,-1.5,54.0,0.0,0.0,10.0,2.000000,-1.0,54.0,0.0,0.0,0.0,0.0,South,South Atlantic,Midwest,West North Central,57.6,0.0,0.0,181102.359,5.4,175,9.6,40.7,0.000,0.0,125328.086,8.3,324,11.9,1,1,0


### Target Encoding for Origin, Destination, Region, Division, and Airline

In [442]:
all_flights = pd.read_csv(DATA_PATH + '/interim/flights_clean.csv')

In [443]:
X_all_flights = all_flights.drop(columns='cancelled')
y_all_flights = all_flights[['cancelled']]

In [444]:
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=42)

# Perform the stratified split
for train_index, test_index in sss.split(X_all_flights, y_all_flights):
    X_train_all, X_test_all = X_all_flights.iloc[train_index], X_all_flights.iloc[test_index]
    y_train_all, y_test_all = y_all_flights.iloc[train_index], y_all_flights.iloc[test_index]

In [482]:
X_train_all['origin'].nunique()

359

In [483]:
X_test_all['origin'].nunique()

359

In [484]:
train_full = pd.concat([X_train_all, y_train_all], axis=1)
test_full = pd.concat([X_test_all, y_test_all], axis=1)

In [485]:
train_full['cancelled'].mean()

0.012931816384261935

In [486]:
test_full['cancelled'].mean()

0.012931744865107059

#### Calculate target encoding using training set of full flights data

In [487]:
departure_window_cancellation_rates = train_full.groupby('departure_window')['cancelled'].mean()
departure_window_cancellation_rates

departure_window
afternoon          0.014409
early afternoon    0.011289
early morning      0.012532
evening            0.017034
late night         0.017726
midday             0.009055
morning            0.009949
night              0.018934
overnight          0.014236
Name: cancelled, dtype: float64

In [488]:
monthly_cancellation_rates = train_full.groupby('month')['cancelled'].mean()
monthly_cancellation_rates

month
1     0.019205
2     0.018028
3     0.012730
4     0.017385
5     0.006326
6     0.020996
7     0.024903
8     0.014887
9     0.012431
10    0.003394
11    0.001366
12    0.004153
Name: cancelled, dtype: float64

In [489]:
origin_cancellation_rates = train_full.groupby('origin')['cancelled'].mean()
origin_cancellation_rates

origin
ABE    0.006612
ABI    0.011352
ABQ    0.009070
ABR    0.018293
ABY    0.001748
         ...   
XNA    0.009790
XWA    0.028523
YAK    0.046512
YKM    0.000000
YUM    0.007494
Name: cancelled, Length: 359, dtype: float64

In [490]:
destination_cancellation_rates = train_full.groupby('dest')['cancelled'].mean()
destination_cancellation_rates

dest
ABE    0.005886
ABI    0.012605
ABQ    0.007965
ABR    0.019920
ABY    0.003454
         ...   
XNA    0.009275
XWA    0.024555
YAK    0.016327
YKM    0.003636
YUM    0.007544
Name: cancelled, Length: 359, dtype: float64

In [491]:
airline_mkt_cancellation_rate = train_full.groupby('airline_mkt')['cancelled'].mean()
airline_mkt_cancellation_rate

airline_mkt
Alaska Airlines Inc.      0.007445
Allegiant Air             0.007053
American Airlines Inc.    0.011700
Delta Air Lines Inc.      0.012278
Frontier Airlines Inc.    0.021332
Hawaiian Airlines Inc.    0.013116
JetBlue Airways           0.021130
Southwest Airlines Co.    0.010010
Spirit Air Lines          0.016926
United Air Lines Inc.     0.017033
Name: cancelled, dtype: float64

In [492]:
airline_ops_cancellation_rate = train_full.groupby('airline_ops')['cancelled'].mean()
airline_ops_cancellation_rate

airline_ops
Air Wisconsin Airlines Corp                0.013602
Alaska Airlines Inc.                       0.008085
Allegiant Air                              0.007053
American Airlines Inc.                     0.010594
CommuteAir LLC dba CommuteAir              0.014637
Delta Air Lines Inc.                       0.010113
Endeavor Air Inc.                          0.021155
Envoy Air                                  0.009625
Frontier Airlines Inc.                     0.021332
GoJet Airlines LLC d/b/a United Express    0.032108
Hawaiian Airlines Inc.                     0.013116
Horizon Air                                0.007436
JetBlue Airways                            0.021130
Mesa Airlines Inc.                         0.020428
PSA Airlines Inc.                          0.012742
Piedmont Airlines                          0.006884
Republic Airline                           0.027701
SkyWest Airlines Inc.                      0.012081
Southwest Airlines Co.                     0.010010


In [493]:
regions_origion_cancellation_rate = train_full.groupby('region_origin')['cancelled'].mean()
regions_origion_cancellation_rate

region_origin
Midwest           0.012725
Northeast         0.021887
South             0.012221
US Territories    0.012283
West              0.010179
Name: cancelled, dtype: float64

In [494]:
division_origion_cancellation_rate = train_full.groupby('division_origin')['cancelled'].mean()
division_origion_cancellation_rate

division_origin
East North Central    0.012896
East South Central    0.011565
Middle Atlantic       0.022573
Mountain              0.011380
New England           0.019637
Pacific               0.009230
South Atlantic        0.011965
US Territories        0.012283
West North Central    0.012351
West South Central    0.012901
Name: cancelled, dtype: float64

In [495]:
regions_dest_cancellation_rate = train_full.groupby('region_dest')['cancelled'].mean()
regions_dest_cancellation_rate

region_dest
Midwest           0.012802
Northeast         0.021887
South             0.012190
US Territories    0.010029
West              0.010230
Name: cancelled, dtype: float64

In [496]:
division_dest_cancellation_rate = train_full.groupby('division_dest')['cancelled'].mean()
division_dest_cancellation_rate

division_dest
East North Central    0.013197
East South Central    0.011201
Middle Atlantic       0.022624
Mountain              0.011496
New England           0.019466
Pacific               0.009229
South Atlantic        0.011812
US Territories        0.010029
West North Central    0.011938
West South Central    0.013203
Name: cancelled, dtype: float64

### Apply target encoding to modeling dataset

In [497]:
# target encode the training dataset
train['dep_window_encoded'] = train['departure_window'].map(departure_window_cancellation_rates)
train['month_encoded'] = train['month'].map(monthly_cancellation_rates)
train['origin_encoded'] = train['origin'].map(origin_cancellation_rates)
train['dest_encoded'] = train['dest'].map(destination_cancellation_rates)
train['airline_mkt_encoded'] = train['airline_mkt'].map(airline_mkt_cancellation_rate)
train['airline_ops_encoded'] = train['airline_ops'].map(airline_ops_cancellation_rate)
train['origin_region_encoded'] = train['region_origin'].map(regions_origion_cancellation_rate)
train['origin_division_encoded'] = train['division_origin'].map(division_origion_cancellation_rate)
train['dest_region_encoded'] = train['region_dest'].map(regions_dest_cancellation_rate)
train['dest_division_encoded'] = train['division_dest'].map(division_dest_cancellation_rate)

In [498]:
train.head()

,quarter,month,day_of_month,day_of_week,hour_of_day,origin,dest,scheduled_elapsed_time,distance,code_share_flight,scheduled_departure_datetime,scheduled_arrival_datetime,airline_mkt,airline_ops,is_holiday,departure_window,dep_delay_mean_10D,dep_delay_median_10D,dep_delay_max_10D,cancelled_sum_10D,div_airport_landings_sum_10D,n_flights_10D,dep_delay_mean_30D,dep_delay_median_30D,dep_delay_max_30D,cancelled_sum_30D,div_airport_landings_sum_30D,cancelled_sum_90D,div_airport_landings_sum_90D,region_origin,division_origin,region_dest,division_dest,temperature_2m_origin,precipitation_origin,snow_depth_origin,visibility_origin,wind_speed_10m_origin,wind_direction_10m_origin,wind_gusts_10m_origin,temperature_2m_dest,precipitation_dest,snow_depth_dest,visibility_dest,wind_speed_10m_dest,wind_direction_10m_dest,wind_gusts_10m_dest,origin_weather_impact_category,dest_weather_impact_category,cancelled,dep_window_encoded,month_encoded,origin_encoded,dest_encoded,airline_mkt_encoded,airline_ops_encoded,origin_region_encoded,origin_division_encoded,dest_region_encoded,dest_division_encoded
783949,2,4,30,7,7,IAH,BOS,288.0,1597.0,0,2023-04-30 07:42:00,2023-04-30 12:30:00,United Air Lines Inc.,United Air Lines Inc.,0,morning,2.000000,-3.0,29.0,0.0,0.0,10.0,9.700000,-2.0,203.0,0.0,0.0,0.0,0.0,South,West South Central,Northeast,New England,55.7,0.0,0.0,51509.188,2.3,241,12.8,49.5,0.016,0.0,656.168,15.2,36,25.1,1,2,0,0.009949,0.017385,0.011403,0.018848,0.017033,0.014099,0.012221,0.012901,0.021887,0.019466
1424121,1,3,31,5,20,IAH,EWR,277.0,1400.0,0,2023-03-31 20:48:00,2023-04-01 01:25:00,United Air Lines Inc.,United Air Lines Inc.,0,night,20.545455,13.0,76.0,0.0,0.0,11.0,11.562500,2.5,76.0,0.0,0.0,0.0,0.0,South,West South Central,Northeast,Middle Atlantic,76.1,0.0,0.0,67257.219,7.9,175,19.2,53.0,0.000,0.0,76115.484,2.6,52,4.9,1,1,0,0.018934,0.012730,0.011403,0.030049,0.017033,0.014099,0.012221,0.012901,0.021887,0.022624
1135132,1,1,30,1,14,SFO,JFK,502.0,2586.0,0,2023-01-30 14:00:00,2023-01-30 22:22:00,Delta Air Lines Inc.,Delta Air Lines Inc.,0,early afternoon,-1.166667,-4.5,9.0,0.0,0.0,6.0,-0.250000,-1.5,16.0,0.0,0.0,0.0,0.0,West,Pacific,Northeast,Middle Atlantic,51.9,0.0,0.0,192257.219,6.1,86,13.2,42.5,0.000,0.0,64960.629,2.9,261,9.2,1,1,0,0.011289,0.019205,0.010708,0.024075,0.012278,0.010113,0.010179,0.009230,0.021887,0.022624
19004,3,8,14,1,18,IAH,MCO,204.0,854.0,0,2023-08-14 18:20:00,2023-08-14 21:44:00,United Air Lines Inc.,United Air Lines Inc.,0,evening,8.100000,2.0,49.0,0.0,0.0,10.0,35.428571,4.5,209.0,2.0,0.0,5.0,0.0,South,West South Central,South,South Atlantic,102.9,0.0,0.0,190288.719,0.2,90,4.0,82.6,0.000,0.0,61679.789,3.0,163,5.8,1,1,0,0.017034,0.014887,0.011403,0.014403,0.017033,0.014099,0.012221,0.012901,0.012190,0.011812
1109133,4,11,30,4,13,ATL,MSP,94.0,907.0,0,2023-11-30 13:48:00,2023-11-30 15:22:00,Delta Air Lines Inc.,Delta Air Lines Inc.,0,early afternoon,7.900000,-1.5,54.0,0.0,0.0,10.0,2.000000,-1.0,54.0,0.0,0.0,0.0,0.0,South,South Atlantic,Midwest,West North Central,57.6,0.0,0.0,181102.359,5.4,175,9.6,40.7,0.000,0.0,125328.086,8.3,324,11.9,1,1,0,0.011289,0.001366,0.008207,0.012289,0.012278,0.010113,0.012221,0.011965,0.012802,0.011938


In [499]:
# Check correlation between region and division columns
region_division_corr = train[['origin_region_encoded', 'origin_division_encoded', 'dest_region_encoded', 'dest_division_encoded']].corr()
region_division_corr

,origin_region_encoded,origin_division_encoded,dest_region_encoded,dest_division_encoded
origin_region_encoded,1.000000,0.977484,-0.015061,-0.038244
origin_division_encoded,0.977484,1.000000,-0.035904,-0.061465
dest_region_encoded,-0.015061,-0.035904,1.000000,0.971644
dest_division_encoded,-0.038244,-0.061465,0.971644,1.000000


Origin and destination are highly correlated, I'll start by only keeping the more granular data stored in division instead of both features. 

In [500]:
# target encode the testing data
test['dep_window_encoded'] = test['departure_window'].map(departure_window_cancellation_rates)
test['month_encoded'] = test['month'].map(monthly_cancellation_rates)
test['origin_encoded'] = test['origin'].map(origin_cancellation_rates)
test['dest_encoded'] = test['dest'].map(destination_cancellation_rates)
test['airline_mkt_encoded'] = test['airline_mkt'].map(airline_mkt_cancellation_rate)
test['airline_ops_encoded'] = test['airline_ops'].map(airline_ops_cancellation_rate)
test['origin_region_encoded'] = test['region_origin'].map(regions_origion_cancellation_rate)
test['origin_division_encoded'] = test['division_origin'].map(division_origion_cancellation_rate)
test['dest_region_encoded'] = test['region_dest'].map(regions_dest_cancellation_rate)
test['dest_division_encoded'] = test['division_dest'].map(division_dest_cancellation_rate)

In [501]:
test.head()

,quarter,month,day_of_month,day_of_week,hour_of_day,origin,dest,scheduled_elapsed_time,distance,code_share_flight,scheduled_departure_datetime,scheduled_arrival_datetime,airline_mkt,airline_ops,is_holiday,departure_window,dep_delay_mean_10D,dep_delay_median_10D,dep_delay_max_10D,cancelled_sum_10D,div_airport_landings_sum_10D,n_flights_10D,dep_delay_mean_30D,dep_delay_median_30D,dep_delay_max_30D,cancelled_sum_30D,div_airport_landings_sum_30D,cancelled_sum_90D,div_airport_landings_sum_90D,region_origin,division_origin,region_dest,division_dest,temperature_2m_origin,precipitation_origin,snow_depth_origin,visibility_origin,wind_speed_10m_origin,wind_direction_10m_origin,wind_gusts_10m_origin,temperature_2m_dest,precipitation_dest,snow_depth_dest,visibility_dest,wind_speed_10m_dest,wind_direction_10m_dest,wind_gusts_10m_dest,origin_weather_impact_category,dest_weather_impact_category,cancelled,dep_window_encoded,month_encoded,origin_encoded,dest_encoded,airline_mkt_encoded,airline_ops_encoded,origin_region_encoded,origin_division_encoded,dest_region_encoded,dest_division_encoded
196803,3,9,13,3,9,JFK,SFO,208.0,2586.0,0,2023-09-13 09:00:00,2023-09-13 12:28:00,Alaska Airlines Inc.,Alaska Airlines Inc.,0,morning,17.10,-2.0,209.0,0.0,0.0,20.0,3.559322,-6.0,209.0,1.0,0.0,1.0,0.0,Northeast,Middle Atlantic,West,Pacific,72.5,0.0,0.0,53149.605,10.2,220,13.4,70.3,0.0,0.0,70209.977,5.5,63,5.4,1,1,0,0.009949,0.012431,0.023118,0.010830,0.007445,0.008085,0.021887,0.022573,0.010230,0.009229
412906,4,12,16,6,23,SEA,BOS,496.0,2496.0,0,2023-12-16 23:59:00,2023-12-17 08:15:00,JetBlue Airways,JetBlue Airways,0,late night,35.80,4.0,209.0,0.0,0.0,10.0,25.206897,0.0,209.0,0.0,0.0,0.0,0.0,West,Pacific,Northeast,New England,35.5,0.0,0.0,51837.270,2.5,5,5.6,43.6,0.0,0.0,56758.531,7.1,103,15.0,1,1,0,0.017726,0.004153,0.006189,0.018848,0.021130,0.021130,0.010179,0.009230,0.021887,0.019466
788628,2,4,10,1,9,LAS,DEN,175.0,628.0,0,2023-04-10 09:42:00,2023-04-10 12:37:00,United Air Lines Inc.,United Air Lines Inc.,0,morning,0.15,-4.5,57.0,0.0,0.0,20.0,4.819672,-5.0,209.0,0.0,0.0,1.0,0.0,West,Mountain,West,Mountain,76.2,0.0,0.0,262795.281,4.7,55,11.2,70.0,0.0,0.0,295275.594,1.9,69,7.6,1,1,0,0.009949,0.017385,0.011676,0.015431,0.017033,0.014099,0.010179,0.011380,0.010230,0.011496
34710,3,8,24,4,15,DCA,FLL,165.0,899.0,0,2023-08-24 15:00:00,2023-08-24 17:45:00,JetBlue Airways,JetBlue Airways,0,afternoon,3.10,-1.0,33.0,0.0,0.0,10.0,32.931034,11.0,209.0,2.0,0.0,4.0,0.0,South,South Atlantic,South,South Atlantic,79.9,0.0,0.0,74146.984,6.9,159,9.4,83.5,0.0,0.0,51509.188,11.5,12,21.9,1,1,0,0.014409,0.014887,0.016129,0.018121,0.021130,0.021130,0.012221,0.011965,0.012190,0.011812
199957,3,9,28,4,8,IAH,SEA,165.0,1874.0,0,2023-09-28 08:30:00,2023-09-28 11:15:00,Alaska Airlines Inc.,Alaska Airlines Inc.,0,morning,-0.60,-8.5,83.0,0.0,0.0,20.0,-2.366667,-6.0,83.0,0.0,0.0,0.0,0.0,South,West South Central,West,Pacific,80.0,0.0,0.0,51181.102,4.9,355,6.0,54.9,0.0,0.0,49868.766,9.1,216,17.2,1,1,0,0.009949,0.012431,0.011403,0.007150,0.007445,0.008085,0.012221,0.012901,0.010230,0.009229


In [502]:
# drop categorical columns
cols_to_drop = ['month', 'departure_window', 'origin', 'dest', 'airline_mkt', 'airline_ops', 'region_origin', 'division_origin', 'region_dest', 'division_dest', 'origin_region_encoded', 'dest_region_encoded']

train = train.drop(columns=cols_to_drop)
test = test.drop(columns=cols_to_drop)

In [503]:
# drop datetime columns
train = train.drop(columns=['scheduled_departure_datetime', 'scheduled_arrival_datetime'])
test = test.drop(columns=['scheduled_departure_datetime', 'scheduled_arrival_datetime'])

In [504]:
train.head()

,quarter,day_of_month,day_of_week,hour_of_day,scheduled_elapsed_time,distance,code_share_flight,is_holiday,dep_delay_mean_10D,dep_delay_median_10D,dep_delay_max_10D,cancelled_sum_10D,div_airport_landings_sum_10D,n_flights_10D,dep_delay_mean_30D,dep_delay_median_30D,dep_delay_max_30D,cancelled_sum_30D,div_airport_landings_sum_30D,cancelled_sum_90D,div_airport_landings_sum_90D,temperature_2m_origin,precipitation_origin,snow_depth_origin,visibility_origin,wind_speed_10m_origin,wind_direction_10m_origin,wind_gusts_10m_origin,temperature_2m_dest,precipitation_dest,snow_depth_dest,visibility_dest,wind_speed_10m_dest,wind_direction_10m_dest,wind_gusts_10m_dest,origin_weather_impact_category,dest_weather_impact_category,cancelled,dep_window_encoded,month_encoded,origin_encoded,dest_encoded,airline_mkt_encoded,airline_ops_encoded,origin_division_encoded,dest_division_encoded
783949,2,30,7,7,288.0,1597.0,0,0,2.000000,-3.0,29.0,0.0,0.0,10.0,9.700000,-2.0,203.0,0.0,0.0,0.0,0.0,55.7,0.0,0.0,51509.188,2.3,241,12.8,49.5,0.016,0.0,656.168,15.2,36,25.1,1,2,0,0.009949,0.017385,0.011403,0.018848,0.017033,0.014099,0.012901,0.019466
1424121,1,31,5,20,277.0,1400.0,0,0,20.545455,13.0,76.0,0.0,0.0,11.0,11.562500,2.5,76.0,0.0,0.0,0.0,0.0,76.1,0.0,0.0,67257.219,7.9,175,19.2,53.0,0.000,0.0,76115.484,2.6,52,4.9,1,1,0,0.018934,0.012730,0.011403,0.030049,0.017033,0.014099,0.012901,0.022624
1135132,1,30,1,14,502.0,2586.0,0,0,-1.166667,-4.5,9.0,0.0,0.0,6.0,-0.250000,-1.5,16.0,0.0,0.0,0.0,0.0,51.9,0.0,0.0,192257.219,6.1,86,13.2,42.5,0.000,0.0,64960.629,2.9,261,9.2,1,1,0,0.011289,0.019205,0.010708,0.024075,0.012278,0.010113,0.009230,0.022624
19004,3,14,1,18,204.0,854.0,0,0,8.100000,2.0,49.0,0.0,0.0,10.0,35.428571,4.5,209.0,2.0,0.0,5.0,0.0,102.9,0.0,0.0,190288.719,0.2,90,4.0,82.6,0.000,0.0,61679.789,3.0,163,5.8,1,1,0,0.017034,0.014887,0.011403,0.014403,0.017033,0.014099,0.012901,0.011812
1109133,4,30,4,13,94.0,907.0,0,0,7.900000,-1.5,54.0,0.0,0.0,10.0,2.000000,-1.0,54.0,0.0,0.0,0.0,0.0,57.6,0.0,0.0,181102.359,5.4,175,9.6,40.7,0.000,0.0,125328.086,8.3,324,11.9,1,1,0,0.011289,0.001366,0.008207,0.012289,0.012278,0.010113,0.011965,0.011938


In [505]:
#train_encoded = pd.get_dummies(train, prefix='dep_window', drop_first=True, dtype='int')
#test_encoded = pd.get_dummies(test, prefix='dep_window', drop_first=True, dtype='int')

train_encoded = train
test_encoded = test

In [506]:
train_encoded.head()

,quarter,day_of_month,day_of_week,hour_of_day,scheduled_elapsed_time,distance,code_share_flight,is_holiday,dep_delay_mean_10D,dep_delay_median_10D,dep_delay_max_10D,cancelled_sum_10D,div_airport_landings_sum_10D,n_flights_10D,dep_delay_mean_30D,dep_delay_median_30D,dep_delay_max_30D,cancelled_sum_30D,div_airport_landings_sum_30D,cancelled_sum_90D,div_airport_landings_sum_90D,temperature_2m_origin,precipitation_origin,snow_depth_origin,visibility_origin,wind_speed_10m_origin,wind_direction_10m_origin,wind_gusts_10m_origin,temperature_2m_dest,precipitation_dest,snow_depth_dest,visibility_dest,wind_speed_10m_dest,wind_direction_10m_dest,wind_gusts_10m_dest,origin_weather_impact_category,dest_weather_impact_category,cancelled,dep_window_encoded,month_encoded,origin_encoded,dest_encoded,airline_mkt_encoded,airline_ops_encoded,origin_division_encoded,dest_division_encoded
783949,2,30,7,7,288.0,1597.0,0,0,2.000000,-3.0,29.0,0.0,0.0,10.0,9.700000,-2.0,203.0,0.0,0.0,0.0,0.0,55.7,0.0,0.0,51509.188,2.3,241,12.8,49.5,0.016,0.0,656.168,15.2,36,25.1,1,2,0,0.009949,0.017385,0.011403,0.018848,0.017033,0.014099,0.012901,0.019466
1424121,1,31,5,20,277.0,1400.0,0,0,20.545455,13.0,76.0,0.0,0.0,11.0,11.562500,2.5,76.0,0.0,0.0,0.0,0.0,76.1,0.0,0.0,67257.219,7.9,175,19.2,53.0,0.000,0.0,76115.484,2.6,52,4.9,1,1,0,0.018934,0.012730,0.011403,0.030049,0.017033,0.014099,0.012901,0.022624
1135132,1,30,1,14,502.0,2586.0,0,0,-1.166667,-4.5,9.0,0.0,0.0,6.0,-0.250000,-1.5,16.0,0.0,0.0,0.0,0.0,51.9,0.0,0.0,192257.219,6.1,86,13.2,42.5,0.000,0.0,64960.629,2.9,261,9.2,1,1,0,0.011289,0.019205,0.010708,0.024075,0.012278,0.010113,0.009230,0.022624
19004,3,14,1,18,204.0,854.0,0,0,8.100000,2.0,49.0,0.0,0.0,10.0,35.428571,4.5,209.0,2.0,0.0,5.0,0.0,102.9,0.0,0.0,190288.719,0.2,90,4.0,82.6,0.000,0.0,61679.789,3.0,163,5.8,1,1,0,0.017034,0.014887,0.011403,0.014403,0.017033,0.014099,0.012901,0.011812
1109133,4,30,4,13,94.0,907.0,0,0,7.900000,-1.5,54.0,0.0,0.0,10.0,2.000000,-1.0,54.0,0.0,0.0,0.0,0.0,57.6,0.0,0.0,181102.359,5.4,175,9.6,40.7,0.000,0.0,125328.086,8.3,324,11.9,1,1,0,0.011289,0.001366,0.008207,0.012289,0.012278,0.010113,0.011965,0.011938


## Scaling Numerical Columns

In [507]:
X_train = train_encoded.drop(columns='cancelled')
y_train = train_encoded['cancelled']

X_test = test_encoded.drop(columns='cancelled')
y_test = test_encoded['cancelled']

In [508]:
X_train.head()

,quarter,day_of_month,day_of_week,hour_of_day,scheduled_elapsed_time,distance,code_share_flight,is_holiday,dep_delay_mean_10D,dep_delay_median_10D,dep_delay_max_10D,cancelled_sum_10D,div_airport_landings_sum_10D,n_flights_10D,dep_delay_mean_30D,dep_delay_median_30D,dep_delay_max_30D,cancelled_sum_30D,div_airport_landings_sum_30D,cancelled_sum_90D,div_airport_landings_sum_90D,temperature_2m_origin,precipitation_origin,snow_depth_origin,visibility_origin,wind_speed_10m_origin,wind_direction_10m_origin,wind_gusts_10m_origin,temperature_2m_dest,precipitation_dest,snow_depth_dest,visibility_dest,wind_speed_10m_dest,wind_direction_10m_dest,wind_gusts_10m_dest,origin_weather_impact_category,dest_weather_impact_category,dep_window_encoded,month_encoded,origin_encoded,dest_encoded,airline_mkt_encoded,airline_ops_encoded,origin_division_encoded,dest_division_encoded
783949,2,30,7,7,288.0,1597.0,0,0,2.000000,-3.0,29.0,0.0,0.0,10.0,9.700000,-2.0,203.0,0.0,0.0,0.0,0.0,55.7,0.0,0.0,51509.188,2.3,241,12.8,49.5,0.016,0.0,656.168,15.2,36,25.1,1,2,0.009949,0.017385,0.011403,0.018848,0.017033,0.014099,0.012901,0.019466
1424121,1,31,5,20,277.0,1400.0,0,0,20.545455,13.0,76.0,0.0,0.0,11.0,11.562500,2.5,76.0,0.0,0.0,0.0,0.0,76.1,0.0,0.0,67257.219,7.9,175,19.2,53.0,0.000,0.0,76115.484,2.6,52,4.9,1,1,0.018934,0.012730,0.011403,0.030049,0.017033,0.014099,0.012901,0.022624
1135132,1,30,1,14,502.0,2586.0,0,0,-1.166667,-4.5,9.0,0.0,0.0,6.0,-0.250000,-1.5,16.0,0.0,0.0,0.0,0.0,51.9,0.0,0.0,192257.219,6.1,86,13.2,42.5,0.000,0.0,64960.629,2.9,261,9.2,1,1,0.011289,0.019205,0.010708,0.024075,0.012278,0.010113,0.009230,0.022624
19004,3,14,1,18,204.0,854.0,0,0,8.100000,2.0,49.0,0.0,0.0,10.0,35.428571,4.5,209.0,2.0,0.0,5.0,0.0,102.9,0.0,0.0,190288.719,0.2,90,4.0,82.6,0.000,0.0,61679.789,3.0,163,5.8,1,1,0.017034,0.014887,0.011403,0.014403,0.017033,0.014099,0.012901,0.011812
1109133,4,30,4,13,94.0,907.0,0,0,7.900000,-1.5,54.0,0.0,0.0,10.0,2.000000,-1.0,54.0,0.0,0.0,0.0,0.0,57.6,0.0,0.0,181102.359,5.4,175,9.6,40.7,0.000,0.0,125328.086,8.3,324,11.9,1,1,0.011289,0.001366,0.008207,0.012289,0.012278,0.010113,0.011965,0.011938


In [509]:
print('Train shape: ', X_train.shape, y_train.shape)
print('Test shape: ', X_test.shape, y_test.shape)

Train shape:  (1045634, 45) (1045634,)
Test shape:  (448130, 45) (448130,)


In [510]:
cat_cols = ['code_share_flight', 'is_holiday']

X_train_cat = X_train[cat_cols]
X_test_cat = X_test[cat_cols]

X_train_num = X_train.drop(columns=cat_cols)
X_test_num = X_test.drop(columns=cat_cols)

In [511]:
X_train_cat.head()

,code_share_flight,is_holiday
783949,0,0
1424121,0,0
1135132,0,0
19004,0,0
1109133,0,0


In [512]:
X_train_num.head()

,quarter,day_of_month,day_of_week,hour_of_day,scheduled_elapsed_time,distance,dep_delay_mean_10D,dep_delay_median_10D,dep_delay_max_10D,cancelled_sum_10D,div_airport_landings_sum_10D,n_flights_10D,dep_delay_mean_30D,dep_delay_median_30D,dep_delay_max_30D,cancelled_sum_30D,div_airport_landings_sum_30D,cancelled_sum_90D,div_airport_landings_sum_90D,temperature_2m_origin,precipitation_origin,snow_depth_origin,visibility_origin,wind_speed_10m_origin,wind_direction_10m_origin,wind_gusts_10m_origin,temperature_2m_dest,precipitation_dest,snow_depth_dest,visibility_dest,wind_speed_10m_dest,wind_direction_10m_dest,wind_gusts_10m_dest,origin_weather_impact_category,dest_weather_impact_category,dep_window_encoded,month_encoded,origin_encoded,dest_encoded,airline_mkt_encoded,airline_ops_encoded,origin_division_encoded,dest_division_encoded
783949,2,30,7,7,288.0,1597.0,2.000000,-3.0,29.0,0.0,0.0,10.0,9.700000,-2.0,203.0,0.0,0.0,0.0,0.0,55.7,0.0,0.0,51509.188,2.3,241,12.8,49.5,0.016,0.0,656.168,15.2,36,25.1,1,2,0.009949,0.017385,0.011403,0.018848,0.017033,0.014099,0.012901,0.019466
1424121,1,31,5,20,277.0,1400.0,20.545455,13.0,76.0,0.0,0.0,11.0,11.562500,2.5,76.0,0.0,0.0,0.0,0.0,76.1,0.0,0.0,67257.219,7.9,175,19.2,53.0,0.000,0.0,76115.484,2.6,52,4.9,1,1,0.018934,0.012730,0.011403,0.030049,0.017033,0.014099,0.012901,0.022624
1135132,1,30,1,14,502.0,2586.0,-1.166667,-4.5,9.0,0.0,0.0,6.0,-0.250000,-1.5,16.0,0.0,0.0,0.0,0.0,51.9,0.0,0.0,192257.219,6.1,86,13.2,42.5,0.000,0.0,64960.629,2.9,261,9.2,1,1,0.011289,0.019205,0.010708,0.024075,0.012278,0.010113,0.009230,0.022624
19004,3,14,1,18,204.0,854.0,8.100000,2.0,49.0,0.0,0.0,10.0,35.428571,4.5,209.0,2.0,0.0,5.0,0.0,102.9,0.0,0.0,190288.719,0.2,90,4.0,82.6,0.000,0.0,61679.789,3.0,163,5.8,1,1,0.017034,0.014887,0.011403,0.014403,0.017033,0.014099,0.012901,0.011812
1109133,4,30,4,13,94.0,907.0,7.900000,-1.5,54.0,0.0,0.0,10.0,2.000000,-1.0,54.0,0.0,0.0,0.0,0.0,57.6,0.0,0.0,181102.359,5.4,175,9.6,40.7,0.000,0.0,125328.086,8.3,324,11.9,1,1,0.011289,0.001366,0.008207,0.012289,0.012278,0.010113,0.011965,0.011938


In [513]:
scaler = StandardScaler()

scaler.fit(X_train_num)

X_train_num_scaled = pd.DataFrame(scaler.transform(X_train_num), columns=X_train_num.columns, index=X_train_num.index)
X_test_num_scaled = pd.DataFrame(scaler.transform(X_test_num), columns=X_test_num.columns, index=X_test_num.index)

In [514]:
X_train_merged = pd.concat([X_train_num_scaled, X_train_cat], axis=1)
X_test_merged = pd.concat([X_test_num_scaled, X_test_cat], axis=1)

In [515]:
X_train_merged

,quarter,day_of_month,day_of_week,hour_of_day,scheduled_elapsed_time,distance,dep_delay_mean_10D,dep_delay_median_10D,dep_delay_max_10D,cancelled_sum_10D,div_airport_landings_sum_10D,n_flights_10D,dep_delay_mean_30D,dep_delay_median_30D,dep_delay_max_30D,cancelled_sum_30D,div_airport_landings_sum_30D,cancelled_sum_90D,div_airport_landings_sum_90D,temperature_2m_origin,precipitation_origin,snow_depth_origin,visibility_origin,wind_speed_10m_origin,wind_direction_10m_origin,wind_gusts_10m_origin,temperature_2m_dest,precipitation_dest,snow_depth_dest,visibility_dest,wind_speed_10m_dest,wind_direction_10m_dest,wind_gusts_10m_dest,origin_weather_impact_category,dest_weather_impact_category,dep_window_encoded,month_encoded,origin_encoded,dest_encoded,airline_mkt_encoded,airline_ops_encoded,origin_division_encoded,dest_division_encoded,code_share_flight,is_holiday
783949,-0.462372,1.627043,1.513039,-1.191404,0.962666,0.793271,-0.637097,-0.433316,-0.848334,-0.321698,-0.026067,-0.485351,-0.255407,-0.357089,1.020906,-0.455582,-0.044988,-0.613999,-0.075349,-0.378699,-0.059103,-0.09847,-0.803722,-1.054689,0.527084,0.099992,-0.765814,0.196989,-0.097696,-1.568529,2.247381,-1.538187,1.803467,-0.240644,4.093543,-0.831642,0.610618,-0.392666,0.727335,0.790950,0.114362,-0.140199,1.415694,0,0
1424121,-1.361143,1.741076,0.513360,1.374386,0.864121,0.490951,0.410658,0.711883,-0.172918,-0.321698,-0.026067,-0.375945,-0.130465,0.046913,-0.854923,-0.455582,-0.044988,-0.613999,-0.075349,0.764768,-0.059103,-0.09847,-0.577887,0.383432,-0.133266,1.007014,-0.568275,-0.060985,-0.097696,-0.492279,-1.006400,-1.377210,-1.049016,-0.240644,-0.240942,1.653785,-0.024426,-0.392666,2.555209,0.790950,0.114362,-0.140199,2.167403,0,0
1135132,-1.361143,1.627043,-1.485998,0.190175,2.879816,2.311011,-0.816002,-0.540678,-1.135745,-0.321698,-0.026067,-0.922973,-0.922882,-0.312199,-1.741141,-0.455582,-0.044988,-0.613999,-0.075349,-0.591698,-0.059103,-0.09847,1.214675,-0.078821,-1.023739,0.156681,-1.160893,-0.060985,-0.097696,-0.651376,-0.928929,0.725541,-0.441804,-0.240644,-0.240942,-0.460922,0.858989,-0.503124,1.580374,-0.491204,-0.702297,-1.015747,2.167403,0,0
19004,0.436398,-0.197480,-1.485998,0.979649,0.210140,-0.346952,-0.292467,-0.075441,-0.560923,-0.321698,-0.026067,-0.485351,1.470541,0.226469,1.109528,1.075347,-0.044988,1.225667,-0.075349,2.266969,-0.059103,-0.09847,1.186446,-1.593984,-0.983718,-1.147163,1.102344,-0.060985,-0.097696,-0.698170,-0.903105,-0.260438,-0.921925,-0.240644,-0.240942,1.128300,0.269856,-0.392666,0.001885,0.790950,0.114362,-0.140199,-0.406655,0,0
1109133,1.335169,1.627043,0.013520,-0.007193,-0.775311,-0.265617,-0.303767,-0.325953,-0.489070,-0.321698,-0.026067,-0.485351,-0.771946,-0.267310,-1.179869,-0.455582,-0.044988,-0.613999,-0.075349,-0.272200,-0.059103,-0.09847,1.054709,-0.258586,-0.133266,-0.353519,-1.262485,-0.060985,-0.097696,0.209624,0.465549,1.359384,-0.060532,-0.240644,-0.240942,-0.460922,-1.574777,-0.900817,-0.343188,-0.491204,-0.702297,-0.363483,-0.376839,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259739,0.436398,-0.539578,-0.986158,-1.191404,0.147429,2.349376,0.712544,-0.361741,1.738364,-0.321698,-0.026067,0.389894,-0.177452,-0.626423,1.109528,-0.455582,-0.044988,-0.613999,-0.075349,0.277113,-0.059103,-0.09847,-0.916639,0.152305,0.647148,0.440125,0.193663,-0.060985,-0.097696,-1.568529,-0.851458,0.896578,-1.260834,-0.240644,-0.240942,-0.831642,1.636376,0.676573,-0.889611,-0.647000,-0.603766,1.466226,-1.021670,0,0
169757,0.436398,-1.451839,1.513039,1.374386,-0.945525,-0.600164,1.475876,0.747671,1.738364,-0.321698,-0.026067,-0.485351,1.255690,0.720248,1.109528,-0.455582,-0.044988,-0.613999,-0.075349,1.050634,-0.059103,-0.09847,0.320746,-0.720839,-0.763601,-0.948752,1.147496,-0.060985,-0.097696,-0.581186,-0.180043,-0.129645,1.012679,-0.240644,-0.240942,1.653785,-0.065277,-0.900817,-0.309602,0.76

In [516]:
X_test_merged

,quarter,day_of_month,day_of_week,hour_of_day,scheduled_elapsed_time,distance,dep_delay_mean_10D,dep_delay_median_10D,dep_delay_max_10D,cancelled_sum_10D,div_airport_landings_sum_10D,n_flights_10D,dep_delay_mean_30D,dep_delay_median_30D,dep_delay_max_30D,cancelled_sum_30D,div_airport_landings_sum_30D,cancelled_sum_90D,div_airport_landings_sum_90D,temperature_2m_origin,precipitation_origin,snow_depth_origin,visibility_origin,wind_speed_10m_origin,wind_direction_10m_origin,wind_gusts_10m_origin,temperature_2m_dest,precipitation_dest,snow_depth_dest,visibility_dest,wind_speed_10m_dest,wind_direction_10m_dest,wind_gusts_10m_dest,origin_weather_impact_category,dest_weather_impact_category,dep_window_encoded,month_encoded,origin_encoded,dest_encoded,airline_mkt_encoded,airline_ops_encoded,origin_division_encoded,dest_division_encoded,code_share_flight,is_holiday
196803,0.436398,-0.311512,-0.486319,-0.796667,0.245974,2.311011,0.216002,-0.361741,1.738364,-0.321698,-0.026067,0.608706,-0.667342,-0.716201,1.109528,0.309882,-0.044988,-0.246066,-0.075349,0.562979,-0.059103,-0.09847,-0.780198,0.974089,0.316973,0.185025,0.408134,-0.060985,-0.097696,-0.576507,-0.257513,-1.266539,-0.978410,-0.240644,-0.240942,-0.831642,-0.065277,1.470294,-0.581203,-1.794510,-1.117895,2.166417,-1.021670,0,0
412906,1.335169,0.030586,1.013199,1.966492,2.826064,2.172895,1.272488,0.067709,1.738364,-0.321698,-0.026067,-0.485351,0.784841,-0.177532,1.109528,-0.455582,-0.044988,-0.613999,-0.075349,-1.510956,-0.059103,-0.09847,-0.799017,-1.003327,-1.834169,-0.920407,-1.098809,-0.060985,-0.097696,-0.768360,0.155665,-0.864099,0.377225,-0.240644,-0.240942,1.319656,-1.194569,-1.221743,0.727335,1.895857,1.554897,-1.015747,1.415694,0,0
788628,-0.462372,-0.653610,-1.485998,-0.796667,-0.049661,-0.693776,-0.741615,-0.540678,-0.445959,-0.321698,-0.026067,0.608706,-0.582794,-0.626423,1.109528,-0.455582,-0.044988,-0.246066,-0.075349,0.770373,-0.059103,-0.09847,2.226227,-0.438351,-1.333903,-0.126763,0.391202,-0.060985,-0.097696,2.633527,-1.187165,-1.206173,-0.667743,-0.240644,-0.240942,-0.831642,0.610618,-0.349301,0.169602,0.790950,0.114362,-0.502955,-0.481954,0,0
34710,0.436398,0.942847,0.013520,0.387544,-0.139247,-0.277894,-0.574950,-0.290166,-0.790852,-0.321698,-0.026067,-0.485351,1.302999,0.810027,1.109528,1.075347,-0.044988,0.857734,-0.075349,0.977766,-0.059103,-0.09847,-0.479085,0.126625,-0.293351,-0.381863,1.153140,-0.060985,-0.097696,-0.843230,1.291906,-1.779651,1.351588,-0.240644,-0.240942,0.402156,0.269856,0.358902,0.608591,1.895857,1.554897,-0.363483,-0.406655,0,0
199957,0.436398,1.398978,0.013520,-0.994035,-0.139247,1.218361,-0.783988,-0.826978,-0.072324,-0.321698,-0.026067,0.608706,-1.064875,-0.716201,-0.751530,-0.455582,-0.044988,-0.613999,-0.075349,0.983372,-0.059103,-0.09847,-0.808427,-0.386990,1.667690,-0.863718,-0.461039,-0.060985,-0.097696,-0.866627,0.672138,0.272795,0.687892,-0.240644,-0.240942,-0.831642,-0.065277,-0.392666,-1.181829,-1.794510,-1.117895,-0.140199,-1.021670,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
505839,-0.462372,0.258651,1.513039,-0.994035,-0.712600,-0.294774,0.430689,-0.433316,1.738364,-0.321698,-0.026067,-0.485351,-0.332661,-0.357089,1.109528,-0.455582,-0.044988,-0.246066,-0.075349,0.316349,-0.059103,-0.09847,-0.257955,-0.284267,0.066840,-0.098419,0.566166,-0.060985,-0.097696,0.892809,1.240259,0.162124,0.151286,-0.240644,-0.240942,-0.831642,1.103247,-0.271924,0.169602,-0.647000,-0.603766,-0.141460,-0.481954,0,0
1423244,-1.361143,0.828814,0.013520,-0.204562,-0.694683,-0.521898,-0.216511,-0.433316,-0.129807,1.246278,-0.026067,-0.594756,-0.078755,-0.357089,0.134688,0.309882,-0.044988,-0.246066,-0.075349,-0.619724,-0.059103,-0.09847,-0.883705,-0.232905,0.056835,-0.764513,-1.234265,-0.060985,-0.097696,-0.459523,0.413902,-1.689102,-0.385319,-0.240644,-0.240942,-1.078897,-0.024426,1.470294,-0.163886,-0.491

In [517]:
y_train

783949     0
1424121    0
1135132    0
19004      0
1109133    0
          ..
259739     0
169757     0
91442      0
82935      0
113566     0
Name: cancelled, Length: 1045634, dtype: int64

# **Feature Selection**

## Variance Inflation Factor
Identify features with high multicolinearity and remove them from the datasets. 

In [518]:
vif_data = pd.DataFrame()
vif_data["Feature"] = X_train_merged.columns
vif_data["VIF"] = [variance_inflation_factor(X_train_merged.values, i) for i in range(X_train_merged.shape[1])]
vif_data = vif_data.sort_values('VIF', ascending=False)
vif_data


,Feature,VIF
6,dep_delay_mean_10D,14.630473
12,dep_delay_mean_30D,11.100128
7,dep_delay_median_10D,7.249110
13,dep_delay_median_30D,6.747961
42,dest_division_encoded,5.547725
38,dest_encoded,5.238014
8,dep_delay_max_10D,5.161111
41,origin_division_encoded,5.142680
37,origin_encoded,4.976623
3,hour_of_day,4.782339


In [519]:
features_reduced = vif_data[vif_data['VIF'] < 5]
features_reduced

,Feature,VIF
37,origin_encoded,4.976623
3,hour_of_day,4.782339
35,dep_window_encoded,4.667848
40,airline_ops_encoded,3.970590
23,wind_speed_10m_origin,3.507525
25,wind_gusts_10m_origin,3.488223
30,wind_speed_10m_dest,3.482827
32,wind_gusts_10m_dest,3.452553
15,cancelled_sum_30D,3.089757
39,airline_mkt_encoded,3.075884


In [520]:
# remove features with VIF >= 5
low_vif_features = features_reduced['Feature'].to_list()

X_train_vif_reduced = X_train_merged[low_vif_features]
X_test_vif_reduced = X_test_merged[low_vif_features]

In [521]:
print(f'Previous train shape: {X_train.shape} \n New train shape {X_train_vif_reduced.shape}. \n')
print(f'Previous test shape: {X_test.shape} \n New test shape {X_test_vif_reduced.shape}.')


Previous train shape: (1045634, 45) 
 New train shape (1045634, 37). 

Previous test shape: (448130, 45) 
 New test shape (448130, 37).


In [522]:
selector = SelectKBest(f_classif, k=10)
selector.fit(X_train_vif_reduced, y_train)
X_train_reduced = selector.transform(X_train_vif_reduced)
feature_scores = selector.scores_
feature_pvalues = selector.pvalues_
selected_features_mask = selector.get_support()
selected_features = X_train_vif_reduced.columns[selected_features_mask]  # Assuming X_train is a DataFrame
# Combine feature names and their scores into a DataFrame
feature_scores_df = pd.DataFrame({
    'Feature': X_train_vif_reduced.columns,
    'Score': feature_scores,
    'P-Value': feature_pvalues
})

# Sort the features by score
feature_scores_df = feature_scores_df.sort_values(by='Score', ascending=False)

# Display the top k features
top_k_features = feature_scores_df.head(10)
top_k_features_mask = top_k_features['Feature'].to_list()


In [523]:
top_k_features_mask

['cancelled_sum_10D',
 'cancelled_sum_30D',
 'cancelled_sum_90D',
 'month_encoded',
 'airline_ops_encoded',
 'origin_weather_impact_category',
 'visibility_dest',
 'dest_weather_impact_category',
 'origin_encoded',
 'quarter']

In [524]:
X_train_reduced = X_train[top_k_features_mask]
X_test_reduced = X_test[top_k_features_mask]

# PCA

In [525]:
"""cat_cols = ['code_share_flight', 'is_holiday', 'dep_window_early afternoon', 'dep_window_early morning', 'dep_window_evening', 'dep_window_late night', 'dep_window_midday', 'dep_window_morning',	'dep_window_night',	'dep_window_overnight']
X_train
X_train_num = X_train.drop(columns=cat_cols)
X_test_num = X_test.drop(columns=cat_cols)
pca = PCA(n_components=0.95)
pca_fit = pca.fit(X_train_num)
X_train_num_pca = pca_fit.transform(X_train_num)
X_test_num_pca = pca_fit.transform(X_test_num)
features = range(pca_fit.n_components_)
plt.bar(features, pca_fit.explained_variance_ratio_)
plt.xlabel('Number of Components')
plt.ylabel('Explained Variance Ratio')
plt.title('Explained Variance by Number of Components')
plt.show()
cols = ['PC1', 'PC2', 'PC3', 'PC4', 'PC5', 'PC6', 'PC7', 'PC8', 'PC9', 'PC10', 'PC11', 'PC12', 'PC13', 'PC14', 'PC15', 'PC16', 'PC17', 'PC18', 'PC19', 'PC20', 'PC21', 'PC22', 'PC23', 'PC24', 'PC25', 'PC26', 'PC27', 'PC28', 'PC29', 'PC30']
loadings = pd.DataFrame(pca_fit.components_.T, columns=cols, index=X_train_num.columns)
loadings.sort_values('PC1', ascending=False)"""

"cat_cols = ['code_share_flight', 'is_holiday', 'dep_window_early afternoon', 'dep_window_early morning', 'dep_window_evening', 'dep_window_late night', 'dep_window_midday', 'dep_window_morning',\t'dep_window_night',\t'dep_window_overnight']\nX_train\nX_train_num = X_train.drop(columns=cat_cols)\nX_test_num = X_test.drop(columns=cat_cols)\npca = PCA(n_components=0.95)\npca_fit = pca.fit(X_train_num)\nX_train_num_pca = pca_fit.transform(X_train_num)\nX_test_num_pca = pca_fit.transform(X_test_num)\nfeatures = range(pca_fit.n_components_)\nplt.bar(features, pca_fit.explained_variance_ratio_)\nplt.xlabel('Number of Components')\nplt.ylabel('Explained Variance Ratio')\nplt.title('Explained Variance by Number of Components')\nplt.show()\ncols = ['PC1', 'PC2', 'PC3', 'PC4', 'PC5', 'PC6', 'PC7', 'PC8', 'PC9', 'PC10', 'PC11', 'PC12', 'PC13', 'PC14', 'PC15', 'PC16', 'PC17', 'PC18', 'PC19', 'PC20', 'PC21', 'PC22', 'PC23', 'PC24', 'PC25', 'PC26', 'PC27', 'PC28', 'PC29', 'PC30']\nloadings = pd.Data

In [526]:
"""X_train_final = np.concatenate([X_train_num_pca, X_train[cat_cols].values], axis=1)
X_test_final = np.concatenate([X_test_num_pca, X_test[cat_cols].values], axis=1)"""

'X_train_final = np.concatenate([X_train_num_pca, X_train[cat_cols].values], axis=1)\nX_test_final = np.concatenate([X_test_num_pca, X_test[cat_cols].values], axis=1)'

# Save train/test splits

In [527]:
X_train_vif_reduced.to_csv(DATA_PATH + '/modeling/X_train.csv', index=False)
X_test_vif_reduced.to_csv(DATA_PATH + '/modeling/X_test.csv', index=False)
X_train_reduced.to_csv(DATA_PATH + '/modeling/X_train_10.csv', index=False)
X_test_reduced.to_csv(DATA_PATH + '/modeling/X_test_10.csv', index=False)
y_train.to_csv(DATA_PATH + '/modeling/y_train.csv', index=False)
y_test.to_csv(DATA_PATH + '/modeling/y_test.csv', index=False)